In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import googlemaps

np.random.seed(1)

# Data preprocessing

## Geocoding

In [ ]:
key = ""
gmaps = googlemaps.Client(key=key)

### Malaysia hospital data

In [ ]:
hospital_df = pd.read_csv("Data Files/Hospital Locations.csv", usecols=["Hospital", "Area"])
hospital_df

,Hospital,Area
0,"Hospital Pakar Sultanah Fatimah, Muar",Johor
1,"Hospital Sultanah Nora Ismail, Batu Pahat",Johor
2,"Hospital Enche Besar Hajjah Kalsom, Kluang",Johor
3,Hospital Segamat,Johor
4,Hospital Pontian,Johor
...,...,...
145,Hospital Dungun,Terengganu
146,Hospital Kemaman,Terengganu
147,Hospital Besut,Terengganu
148,Hospital Hulu Terengganu,Terengganu


Removing duplicate hospitals

In [ ]:
hospital_df = hospital_df.drop_duplicates("Hospital").reset_index(drop=True)
hospital_df

,Hospital,Area
0,"Hospital Pakar Sultanah Fatimah, Muar",Johor
1,"Hospital Sultanah Nora Ismail, Batu Pahat",Johor
2,"Hospital Enche Besar Hajjah Kalsom, Kluang",Johor
3,Hospital Segamat,Johor
4,Hospital Pontian,Johor
...,...,...
144,Hospital Dungun,Terengganu
145,Hospital Kemaman,Terengganu
146,Hospital Besut,Terengganu
147,Hospital Hulu Terengganu,Terengganu


In [ ]:
hospital_df["address"] = hospital_df["Hospital"] + ", " + hospital_df["Area"] + ", Malaysia"

### Geocoding hospitals

In [ ]:
def get_geocode(address):
    geo_result = gmaps.geocode(address)
    
    # Extracting latitude and longitude from the response json
    lat = geo_result[0]["geometry"]["location"]["lat"]
    lng = geo_result[0]["geometry"]["location"]["lng"]
    
    return lat, lng

In [ ]:
hospital_df[["lat","lng"]] = hospital_df.apply(lambda row: get_geocode(row.address), axis='columns', result_type='expand')
hospital_df

,Hospital,Area,address,lat,lng
0,"Hospital Pakar Sultanah Fatimah, Muar",Johor,"Hospital Pakar Sultanah Fatimah, Muar, Johor, ...",2.057115,102.574446
1,"Hospital Sultanah Nora Ismail, Batu Pahat",Johor,"Hospital Sultanah Nora Ismail, Batu Pahat, Joh...",1.837516,102.940997
2,"Hospital Enche Besar Hajjah Kalsom, Kluang",Johor,"Hospital Enche Besar Hajjah Kalsom, Kluang, Jo...",2.038227,103.311296
3,Hospital Segamat,Johor,"Hospital Segamat, Johor, Malaysia",2.493269,102.859063
4,Hospital Pontian,Johor,"Hospital Pontian, Johor, Malaysia",1.498460,103.385207
...,...,...,...,...,...
144,Hospital Dungun,Terengganu,"Hospital Dungun, Terengganu, Malaysia",4.751876,103.414832
145,Hospital Kemaman,Terengganu,"Hospital Kemaman, Terengganu, Malaysia",4.232613,103.421057
146,Hospital Besut,Terengganu,"Hospital Besut, Terengganu, Malaysia",5.729650,102.492570
147,Hospital Hulu Terengganu,Terengganu,"Hospital Hulu Terengganu, Terengganu, Malaysia",5.073216,103.044775


Removing some hospitals that are not connected to any blood centers (via Google Maps driving routes)

In [ ]:
hospital_df[hospital_df.Hospital == "Langkawi Hospital"]

,Hospital,Area
17,Langkawi Hospital,Kedah


In [ ]:
hospital_df[hospital_df.Hospital == "Hospital Angkatan Tentera, Pangkalan TLDM, Lumut"]

,Hospital,Area
84,"Hospital Angkatan Tentera, Pangkalan TLDM, Lumut",Perak


In [ ]:
hospital_df = hospital_df.drop([17, 84], axis = 0).reset_index(drop=True)
hospital_df

,Hospital,Area
0,"Hospital Pakar Sultanah Fatimah, Muar",Johor
1,"Hospital Sultanah Nora Ismail, Batu Pahat",Johor
2,"Hospital Enche Besar Hajjah Kalsom, Kluang",Johor
3,Hospital Segamat,Johor
4,Hospital Pontian,Johor
...,...,...
142,Hospital Dungun,Terengganu
143,Hospital Kemaman,Terengganu
144,Hospital Besut,Terengganu
145,Hospital Hulu Terengganu,Terengganu


In [ ]:
hospital_df.to_csv("Hospital Locations.csv", index=False)

### Geocoding blood centers

In [ ]:
b_centers = ['Hospital Duchess Of Kent', 'Hospital Melaka', 'Hospital Miri',
             'Hospital Pulau Pinang', 'Hospital Queen Elizabeth II',
             'Hospital Raja Perempuan Zainab II',
             'Hospital Raja Permaisuri Bainun', 'Hospital Seberang Jaya',
             'Hospital Seri Manjung', 'Hospital Sibu',
             'Hospital Sultan Haji Ahmad Shah', 'Hospital Sultanah Aminah',
             'Hospital Sultanah Bahiyah', 'Hospital Sultanah Nora Ismail',
             'Hospital Sultanah Nur Zahirah', 'Hospital Taiping',
             'Hospital Tawau', 'Hospital Tengku Ampuan Afzan',
             'Hospital Tengku Ampuan Rahimah', 'Hospital Tuanku Jaafar',
             'Hospital Umum Sarawak', 'Pusat Darah Negara']

In [ ]:
bc_df = pd.DataFrame({"Name": b_centers})
bc_df

,Name
0,Hospital Duchess Of Kent
1,Hospital Melaka
2,Hospital Miri
3,Hospital Pulau Pinang
4,Hospital Queen Elizabeth II
5,Hospital Raja Perempuan Zainab II
6,Hospital Raja Permaisuri Bainun
7,Hospital Seberang Jaya
8,Hospital Seri Manjung
9,Hospital Sibu


In [ ]:
bc_df["address"] = bc_df["Name"] + ", Malaysia"

In [ ]:
bc_df[["lat","lng"]] = bc_df.apply(lambda row: get_geocode(row.address), axis='columns', result_type='expand')

In [ ]:
bc_df.to_csv("Blood Center Locations.csv", index=False)

## Travel time between centers

In [ ]:
hospital_df = pd.read_csv("Data Files/Hospital Locations.csv")
bc_df = pd.read_csv("Data Files/Blood Center Locations.csv")

In [ ]:
hospital_df["coords"] = list(zip(hospital_df["lat"], hospital_df["lng"]))
bc_df["coords"] = list(zip(bc_df["lat"], bc_df["lng"]))

In [ ]:
n_centers = 22
max_elements = 100 # GMaps API has a limit of 100 elements per call
max_dists = int(max_elements/n_centers)
time_df = pd.DataFrame(index=b_centers)

for i in range(0, len(hospital_df), max_dists):
    res = gmaps.distance_matrix(origins=bc_df["coords"],
                                destinations=hospital_df.loc[i:i+max_dists - 1, "coords"],
                                mode="driving")

    rows = []
    for row in res["rows"]:
        rows.append(pd.json_normalize(row["elements"])["duration.value"].values)
        
    new_hospital_df = pd.DataFrame(rows, 
                          index = b_centers,
                          columns = [hospital_df.loc[x, "Hospital"] for x in range(i, min(len(hospital_df), i+max_dists))])

    # Cocatenating the additional distances with the original distance matrix
    time__df = pd.concat([time_df, new_hospital_df], axis = 1)

In [ ]:
time_df = round(time_df/60)

In [ ]:
time_df.head()

,"Hospital Pakar Sultanah Fatimah, Muar","Hospital Sultanah Nora Ismail, Batu Pahat","Hospital Enche Besar Hajjah Kalsom, Kluang",Hospital Segamat,Hospital Pontian,Hospital Kota Tinggi,Hospital Mersing,Hospital Tangkak,"Hospital Temenggung Seri Maharaja Tun Ibrahim, Kulai",Sultan Ismail Hospital,...,"Hospital Tengku Ampuan Jemaah, Sabak Bernam","Hospital Tengku Ampuan Rahimah, Klang",Universiti Putra Malaysia Teaching Hospital,"Universiti Teknologi MARA Teaching Hospital, Puncak Alam","Hospital Sultanah Nur Zahirah, Kuala Terengganu",Hospital Dungun,Hospital Kemaman,Hospital Besut,Hospital Hulu Terengganu,Hospital Setiu
Hospital Duchess Of Kent,6453.0,6432.0,6414.0,6462.0,6375.0,6376.0,6449.0,6448.0,6359.0,6352.0,...,6643.0,6557.0,6529.0,6562.0,6730.0,6677.0,6644.0,6786.0,6696.0,6746.0
Hospital Melaka,51.0,117.0,115.0,107.0,142.0,173.0,195.0,48.0,137.0,150.0,...,210.0,124.0,96.0,129.0,364.0,310.0,278.0,420.0,330.0,380.0
Hospital Miri,5676.0,5655.0,5637.0,5685.0,5598.0,5599.0,5672.0,5672.0,5583.0,5575.0,...,5866.0,5780.0,5752.0,5785.0,5953.0,5900.0,5867.0,6010.0,5920.0,5969.0
Hospital Pulau Pinang,365.0,413.0,411.0,403.0,438.0,469.0,491.0,344.0,434.0,446.0,...,194.0,254.0,261.0,238.0,426.0,458.0,425.0,326.0,431.0,365.0
Hospital Queen Elizabeth II,6080.0,6059.0,6041.0,6089.0,6002.0,6003.0,6076.0,6076.0,5987.0,5979.0,...,6270.0,6184.0,6156.0,6189.0,6357.0,6304.0,6271.0,6414.0,6324.0,6373.0


In [ ]:
time_df.drop("Langkawi Hospital", axis=1, inplace=True)

In [ ]:
time_df.drop("Hospital Angkatan Tentera, Pangkalan TLDM, Lumut", axis=1, inplace=True)

In [ ]:
time_df.to_csv("Data Files/Travel Times.csv")